Testing Guney's `toolbox`

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from toolbox import wrappers

GenRev not found, steiner wont work
Import error: Negex. Using keyword matching instead
Import error: Funcassociate. Make sure that funcassociate is in toolbox!


## Preparations

In [2]:
id_mapping_file = '../../resources/PPI/geneid_to_symbol.txt'
network_guney = wrappers.get_network('../../resources/proximity/data/network/network.sif', only_lcc = True)

I will use Hydroxychloroquine's targets.  See [this page](https://go.drugbank.com/drugs/DB01611) on drugbank.

In [3]:
%%bash
echo TLR7 > ../../results/2021-08-04-guney-tools/Hydroxychloroquine-targets
echo TLR9 >> ../../results/2021-08-04-guney-tools/Hydroxychloroquine-targets

In [4]:
Hydroxychloroquine_targets = wrappers.convert_to_geneid(file_name='../../results/2021-08-04-guney-tools/Hydroxychloroquine-targets', id_type='symbol', id_mapping_file=id_mapping_file)
Hydroxychloroquine_targets

set()


{'51284', '54106'}

## Guney AD genes

In [5]:
%%bash
grep 'alzheimer disease' ../../resources/proximity/data/disease/disease_genes.tsv | \
tr '\t' '\n' | sed -n '/^[0-9]\+/ p' > ../../results/2021-08-04-guney-tools/AD-genes-guney

In [6]:
with open('../../results/2021-08-04-guney-tools/AD-genes-guney') as f:
    AD_genes_guney = f.readlines()
AD_genes_guney = [x.strip('\n') for x in AD_genes_guney]

Making sure that the gene set is a subset of the network's nodes

In [7]:
def remove_genes_notin_network(genes, network):
    oldsize = len(genes)
    newgenes = [y for y in genes if y in network.nodes]
    newsize = len(newgenes)
    restgenes = set(genes).difference(set(newgenes))
    print(oldsize - newsize, 'genes removed from', oldsize)
    return((newgenes, restgenes))

AD_genes_guney, AD_genes_guney_removed = remove_genes_notin_network(AD_genes_guney, network_guney)

7 genes removed from 36


In [8]:
res_guney = wrappers.calculate_proximity(network=network_guney, nodes_from=Hydroxychloroquine_targets, nodes_to=AD_genes_guney)

In [9]:
res_guney

(2.5, 0.08121892330210621, (2.4665, 0.41246545309880195))

## Knowledge based AD genes

In [10]:
AD_genes_knowledge = wrappers.convert_to_geneid(file_name='../../results/2021-07-01-high-conf-ADgenes/AD-genes-knowledge', id_type='symbol', id_mapping_file=id_mapping_file)

{'MT-ND2', 'MT-ND1'}


In [11]:
AD_genes_knowledge, AD_genes_knowledge_removed = remove_genes_notin_network(AD_genes_knowledge, network_guney)

1 genes removed from 24


In [12]:
res_knowledge = wrappers.calculate_proximity(network=network_guney, nodes_from=Hydroxychloroquine_targets, nodes_to=AD_genes_knowledge)

In [13]:
res_knowledge

(3.0, 1.4263778227171104, (2.4445, 0.38944800679936725))

### Changing targets

In [14]:
res_knowledge_prox = wrappers.calculate_proximity(network=network_guney, nodes_from=AD_genes_knowledge[:2], nodes_to=AD_genes_knowledge)

In [15]:
%connect_info

{
  "shell_port": 40677,
  "iopub_port": 45183,
  "stdin_port": 51959,
  "control_port": 59383,
  "hb_port": 52937,
  "ip": "127.0.0.1",
  "key": "baacd117-1e3fc134b602b42211df35cd",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-9700232e-f654-4840-9d63-d4b7a178eaff.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
